In [1]:
import csv
import os
import datetime
from email.utils import parsedate_to_datetime

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [2]:
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
# !pip install --upgrade google-auth-oauthlib
# !pip uninstall google-auth-oauthlib -y
# !pip install google-auth-oauthlib==0.5.3


In [3]:
!pip show google-auth-oauthlib

Name: google-auth-oauthlib
Version: 0.5.3
Summary: Google Authentication Library
Home-page: https://github.com/GoogleCloudPlatform/google-auth-library-python-oauthlib
Author: Google Cloud Platform
Author-email: jonwayne+google-auth@google.com
License: Apache 2.0
Location: C:\ProgramData\anaconda3\Lib\site-packages
Requires: google-auth, requests-oauthlib
Required-by: gspread


In [4]:
# Scopes only for Gmail read
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [5]:
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import os
import json

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def authenticate_gmail():
    creds = None

    # 1. Load from token if exists
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    # 2. If no valid creds, start OAuth flow
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)

        # Manual browser-based login
        auth_url, _ = flow.authorization_url(prompt='consent')
        print("Please go to this URL:\n", auth_url)

        # User pastes the code manually
        code = input("Paste the authorization code here: ").strip()
        flow.fetch_token(code=code)
        creds = flow.credentials

        # Save credentials to token.json
        with open('token.json', 'w') as token_file:
            token_file.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)


In [6]:
def get_mtd_date_range():
    now = datetime.datetime.now()
    start = datetime.datetime(now.year, now.month, 1)
    end = datetime.datetime(now.year, now.month, now.day, 23, 59, 59)
    return start, end

def gmail_query(start_date, end_date):
    start_str = start_date.strftime('%Y/%m/%d')
    end_str = (end_date + datetime.timedelta(days=1)).strftime('%Y/%m/%d')
    return f'after:{start_str} before:{end_str}'

In [7]:

def export_emails_to_csv():
    service = authenticate_gmail()
    start_date, end_date = get_mtd_date_range()
    query = gmail_query(start_date, end_date)

    results = service.users().messages().list(userId='me', q=query, maxResults=500).execute()
    messages = results.get('messages', [])

    rows = [['Date', 'From', 'To', 'Subject', 'Snippet']]
    print(f"Found {len(messages)} messages for query: {query}")

    for msg in messages:
        msg_data = service.users().messages().get(userId='me', id=msg['id'], format='metadata', metadataHeaders=['From', 'To', 'Subject']).execute()
        headers = msg_data['payload'].get('headers', [])

        msg_date = datetime.datetime.fromtimestamp(int(msg_data['internalDate']) / 1000)
        from_ = next((h['value'] for h in headers if h['name'] == 'From'), '')
        to_ = next((h['value'] for h in headers if h['name'] == 'To'), '')
        subject = next((h['value'] for h in headers if h['name'] == 'Subject'), '')
        snippet = msg_data.get('snippet', '').replace('\n', ' ')[:100]

        if start_date <= msg_date <= end_date:
            rows.append([msg_date.strftime('%Y-%m-%d %H:%M'), from_, to_, subject, snippet])

    # Export to CSV
    filename = f'gmail_mtd_export_{datetime.datetime.now().strftime("%Y%m%d")}.csv'
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(rows)

    print(f"Exported {len(rows)-1} messages to {filename}")


In [ ]:
if __name__ == '__main__':
    export_emails_to_csv()

Please go to this URL:
 https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=888988467470-d88gr2h2lq08sa1ee1jbbusepfji8nk1.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=lLNjhK6QnUM2isO9aCbrLPMzQh2JT0&prompt=consent&access_type=offline
